In [41]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import IncrementalPCA
from tabulate import tabulate
import tensorflow as tf
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

## Initial Loading

In [43]:

ddi_fp = "drugbank\drugbank.tab"
ddi = pd.read_csv(ddi_fp, sep='\t')

kaggle_fp = "SMILES-Kaggle\chembl_22_clean_1576904_sorted_std_final.smi"
smiles = pd.read_csv(kaggle_fp, sep='\t')

drug_names_fp = "drugs.txt"
drug_names = pd.read_csv(drug_names_fp, sep='\t')

ddi["Y"] = ddi["Y"].astype("category")
ddi["Map"] = ddi["Map"].astype("category")

#counting interaction types for potential later weighting
interaction_counts = pd.DataFrame(ddi['Y'].value_counts().rename_axis('value').reset_index(name='count')).sort_values(by='count', ascending=False)
interaction_counts['row_num'] = interaction_counts.index + 1
interaction_counts['log_count'] = np.log(interaction_counts['count'])

#listing longer explanations of interaction types for later use
interaction_types = ddi[['Y','Map']].drop_duplicates(subset=['Y'])

#remove longer name of interaction type from main dataset
ddi = ddi.drop("Map",axis=1)

ddi.head(10)

,ID1,ID2,Y,X1,X2
0,DB04571,DB00460,1,CC1=CC2=CC3=C(OC(=O)C=C3C)C(C)=C2O1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
1,DB00855,DB00460,1,NCC(=O)CCC(O)=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
2,DB09536,DB00460,1,O=[Ti]=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
3,DB01600,DB00460,1,CC(C(O)=O)C1=CC=C(S1)C(=O)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
4,DB09000,DB00460,1,CC(CN(C)C)CN1C2=CC=CC=C2SC2=C1C=C(C=C2)C#N,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
5,DB11630,DB00460,1,OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
6,DB00553,DB00460,1,COC1=C2OC(=O)C=CC2=CC2=C1OC=C2,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
7,DB06261,DB00460,1,[H]N([H])CC(=O)CCC(=O)OCCCCCC,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
8,DB01878,DB00460,1,O=C(C1=CC=CC=C1)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
9,DB00140,DB00460,1,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...


In [44]:
ddi.shape

(191808, 5)

In [45]:
#quick function to turn a list of size 1 lists of strings into a list of strings, for later use
def delist(list_of_lists):
    list_of_strings = []
    for inner_list in list_of_lists:
        string = inner_list[0]
        list_of_strings.append(string)
    return list_of_strings

## Preprocessing

In [47]:

# counting drugs by number of mentions in database
old = pd.DataFrame()
old["total"] = ddi['ID1'].value_counts()
old = old.reset_index()
old.columns = ['ID', 'count'] 
new = pd.DataFrame()
new["total"] = ddi['ID2'].value_counts()
new = new.reset_index()
new.columns = ['ID', 'count'] 
drug_counts = pd.merge(old,new,how='outer',on='ID').fillna(0)
drug_counts['total'] = drug_counts['count_x'] + drug_counts['count_y']

drug_counts = drug_counts.sort_values(by='total')
drug_counts_one = pd.DataFrame(drug_counts[drug_counts['total']==1]['ID'])

#removing drugs only in database once
ddi_proc = ddi[ ~ddi['ID1'].isin(drug_counts_one['ID'])]
ddi_proc = ddi_proc[ ~ddi_proc['ID2'].isin(drug_counts_one['ID'])]

#removing one particular drug with a problematic SMILES code
ddi_proc = ddi_proc[ddi_proc['X1']!="OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1"]


In [48]:
#create main datasets

data = dc.data.NumpyDataset(X=ddi_proc[['X1','X2']], y=ddi[['Y']])
df = data.to_dataframe()
df = df.sample(frac=1).reset_index(drop=True)

X_one = delist(df[["X1"]].values.tolist())
X_two = delist(df[["X2"]].values.tolist())


search_string = "nan"

count = X_one.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_one.")
count = X_two.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_two.")

## Featurization and Dimensionality Reduction

In [50]:
#featurize using circular fingerprint

cf_featurizer = dc.feat.CircularFingerprint()

def circular_fingerprint(smiles):
    try:
        mol = cf_featurizer(smiles)
        return mol
    except Exception as e:
        print(f"Error fingerprinting {smiles}: {e}")
        return None  # Skipping invalid SMILES


#other way to featurize a molecule
#cm_featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

#example process with just 10000 rows
df_example = df.head(10000)
df_example = df_example.reset_index()
df_example.rename(columns={'index':'col'},  inplace=True)

df_example_cf = pd.DataFrame()
df_example_cf['x1_cf'] = df_example['X1'].apply(circular_fingerprint)
df_example_cf['x2_cf'] = df_example['X2'].apply(circular_fingerprint)
df_example_cf['col'] = df_example_cf.index
df_example_cf.dropna(subset=['x1_cf', 'x2_cf'], inplace=True)


df_example_x1_cf = pd.DataFrame(delist(df_example_cf['x1_cf']))
df_example_x1_cf.rename(columns=lambda x: "x1_cf_"+str(x+1), inplace=True)
df_example_x1_cf['col'] = df_example_cf['col']

df_example_x2_cf = pd.DataFrame(delist(df_example_cf['x2_cf']))
df_example_x2_cf.rename(columns=lambda x: "x2_cf_"+str(x+1), inplace=True)
df_example_x2_cf['col'] = df_example_cf['col']


df_example = df_example.merge(df_example_x1_cf, on="col")
df_example = df_example.merge(df_example_x2_cf, on="col")
df_example.dropna(inplace=True)

df_example

[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerator
[11:37:19] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerator
[11:37:35] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerator
[11:37:39] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerator
[11:37:58] DEPRECATION WARNING: please use MorganGenerat

Error fingerprinting nan: 'float' object is not iterable


[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:00] DEPRECATION WARNING: please use MorganGenerator
[11:38:01] DEPRECATION WARNING: please use MorganGenerator
[11:38:01] DEPRECATION WARNING: please use MorganGenerator
[11:38:01] DEPRECATION WARNING: please use MorganGenerat

,col,X1,X2,y,w,ids,x1_cf_1,x1_cf_2,x1_cf_3,x1_cf_4,...,x2_cf_2039,x2_cf_2040,x2_cf_2041,x2_cf_2042,x2_cf_2043,x2_cf_2044,x2_cf_2045,x2_cf_2046,x2_cf_2047,x2_cf_2048
0,0,[Gd+3].OC(=O)CN(CCN(CCN(CC([O-])=O)CC([O-])=O)...,COC1=CC=CC=C1OCC(O)CN1CCN(CC(=O)NC2=C(C)C=CC=C...,20,1.0,23335,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,COC1=CC2=C(NC=C2CCNC(C)=O)C=C1,CN1C2=C(C=C(Cl)C=C2)C(=NCC1=O)C1=CC=CC=C1F,49,1.0,119809,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,[H][C@@]12C[C@]1([H])[C@@]1(C)C(=CC2=O)C(Cl)=C...,COC1=CC2=C(C=C1)C=C(CCC(C)=O)C=C2,75,1.0,187399,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,CC(C)C[C@H](CN)CC(O)=O,COC1=CC=C(C=C1)C(CN(C)C)C1(O)CCCCC1,49,1.0,92212,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,CNCCCC12CCC(C3=CC=CC=C13)C1=CC=CC=C21,NC(=N)NC(=O)CC1=C(Cl)C=CC=C1Cl,49,1.0,71647,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,9993,CCOC1=CC=CC=C1OCCN[C@H](C)CC1=CC(=C(OC)C=C1)S(...,CN[C@@H](C)[C@H](O)C1=CC=CC=C1,5,1.0,5907,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9992,9994,CNC(=O)C1=CC(OC2=CC(F)=C(NC(=O)NC3=CC=C(Cl)C(=...,OC(CNCC(O)C1CCC2=C(O1)C=CC(F)=C2)C1CCC2=C(O1)C...,54,1.0,131137,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9993,9995,[H][C@@]12CC[C@H](C(=O)NC(C)(C)C)[C@@]1(C)CC[C...,[H][C@@]12CCO[C@]1([H])OC[C@@H]2OC(=O)N[C@@H](...,47,1.0,62590,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9994,9996,CCCCN1CCCC[C@H]1C(=O)NC1=C(C)C=CC=C1C,BrC1=C(NC2=NCCN2)C=CC2=NC=CN=C12,16,1.0,13608,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
#Basic dimensionality reduction down to 100 components from 4096

df_X = df_example.iloc[:,6:]
df_Y = to_categorical(df_example["y"])

n_components = 100
batch_size = 200

ipca = IncrementalPCA(n_components=n_components)

for i in range(0, df_X.shape[0], batch_size):
    X_batch = df_X[i:i + batch_size]
    ipca.partial_fit(X_batch)

X_transformed = ipca.transform(df_X)

#prove that X's shape has changed
#print("Original shape:", df_X.shape)
#print("Transformed shape:", X_transformed.shape)

df_X_proc = pd.DataFrame(X_transformed)


## Initial basic model implementation

In [53]:
#basic/rough neural network implementation

def calc_layers(X_size, Y_size):
    layers = [X_size+1]
    layer = 2
    while layer <= X_size:
        layer = int(layer * 2)
    layers.append(layer)
    if X_size > Y_size:
        while layer / 2 > Y_size and layer > 2:
            layer = layer / 2
            layers.append(int(layer))
    layers.append(Y_size)
    return layers
        
def DNN(X, Y, Epochs, batchsize, layernum):

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=27)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #define layers and nodes in each layer
    input = len(X.columns)
    output = Y.shape[1]
    if layernum=='many':
        layers = calc_layers(input,output)
    elif type(layernum)==int:
        layers = [input]
        for i in range(layernum, 1, -1):
            layer = int(round((i * (input + output) / (layernum+1)), 0))
            if layer > output:
                layers.append(layer)
        layers.append(output)
    else:
        print(f"incorrect layernum {layernum}")
        return None

    model = keras.models.Sequential()

    model.add(Dense(layers[0], activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    for layer_size in layers[1:-1]:
        model.add(Dense(layer_size, activation='relu'))
        model.add(keras.layers.Dropout(0.2))
        
    model.add(Dense(layers[-1], activation='softmax'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy','AUC','precision','recall'])
    
    model.fit(X_train, y_train, epochs=Epochs, batch_size=batchsize, validation_split=0.1)
    
    full_loss, full_accuracy, full_AUC, full_precision, full_recall = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {full_accuracy}")
    print(f"Test Loss: {full_loss}")
    return model

In [54]:
dr_model = DNN(df_X_proc, df_Y, 5, 1000, 1)

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - AUC: 0.5877 - accuracy: 0.0148 - loss: 4.5112 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_AUC: 0.7501 - val_accuracy: 0.0487 - val_loss: 4.0552 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - AUC: 0.7611 - accuracy: 0.0667 - loss: 3.9965 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_AUC: 0.8594 - val_accuracy: 0.1112 - val_loss: 3.6179 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - AUC: 0.8617 - accuracy: 0.1422 - loss: 3.5454 - precision: 0.1669 - recall: 2.3040e-04 - val_AUC: 0.8990 - val_accuracy: 0.1762 - val_loss: 3.2625 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - AUC: 0.9027 - accuracy: 0.2121 - loss: 3.1753 - precision: 0.4314 - recall: 0.0025 - val_AUC: 0.9133 - val_accuracy: 0.2350 - val_loss: 2.9861 - val_precision: 0.4286 - val_recall: 0.0037
Epo

In [55]:
#comparison to without Dimensionality reduction

full_model = DNN(df_X, df_Y, 5, 1000, 1)

#calc_layers(len(df_X.iloc[:,1:].columns), df_Y.shape[1])

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 655ms/step - AUC: 0.7585 - accuracy: 0.3032 - loss: 3.7583 - precision: 0.4667 - recall: 0.2564 - val_AUC: 0.8692 - val_accuracy: 0.5038 - val_loss: 3.4819 - val_precision: 0.5433 - val_recall: 0.4787
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - AUC: 0.9618 - accuracy: 0.7337 - loss: 1.1797 - precision: 0.7641 - recall: 0.7098 - val_AUC: 0.8532 - val_accuracy: 0.4787 - val_loss: 3.7737 - val_precision: 0.5014 - val_recall: 0.4575
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 494ms/step - AUC: 0.9898 - accuracy: 0.8592 - loss: 0.5056 - precision: 0.8771 - recall: 0.8416 - val_AUC: 0.8601 - val_accuracy: 0.4925 - val_loss: 3.7864 - val_precision: 0.5165 - val_recall: 0.4700
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 488ms/step - AUC: 0.9969 - accuracy: 0.9279 - loss: 0.2522 - precision: 0.9378 - recall: 0.9196 - val_AUC: 0.8541 - val_accuracy: 0.4888 - val_loss: 3.9385 - val_precision: 0.5108 - val_recall: 0.4725
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 486ms/

This demonstrates that dimensionality reduction is counterproductive to a neural network in this case.

## Adding in other variables

In [72]:
#code from Beth Farr
def compute_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return None

        # Computing Molecular Descriptors
        mol_wt = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        h_donors = Descriptors.NumHDonors(mol)
        h_acceptors = Descriptors.NumHAcceptors(mol)
        tpsa = Descriptors.TPSA(mol)
        return [mol_wt, logp, h_donors, h_acceptors, tpsa]

    except Exception as e:
        print(f"Error computing features for {smiles}: {e}")
        return None  # Skipping invalid SMILES

# Using function to X1 and X2 to extract features
df_feat = df_example
df_feat['features_X1'] = df_example['X1'].apply(compute_features)
df_feat['features_X2'] = df_example['X2'].apply(compute_features)
df_feat = df_feat.dropna(subset=['features_X1', 'features_X2'])
features_X1_df = pd.DataFrame(df_feat['features_X1'].tolist(), columns=['MolWt_X1', 'LogP_X1', 'NumHDonors_X1', 'NumHAcceptors_X1', 'TPSA_X1'])
features_X1_df['col'] = df_feat['col']
features_X2_df = pd.DataFrame(df_feat['features_X2'].tolist(), columns=['MolWt_X2', 'LogP_X2', 'NumHDonors_X2', 'NumHAcceptors_X2', 'TPSA_X2'])
features_X2_df['col'] = df_feat['col']

df_feat = pd.merge(df_feat, pd.merge(features_X1_df, features_X2_df,on='col'), on='col')
df_feat = df_feat.drop(['X1', 'X2', 'w', 'features_X1', 'features_X2'], axis=1)

df_feat_X = df_feat.iloc[:,3:]
df_feat_Y = to_categorical(df_feat["y"])

feat_model = DNN(df_feat_X, df_feat_Y, 5, 1000, 1)


Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 479ms/step - AUC: 0.7927 - accuracy: 0.3187 - loss: 3.5989 - precision: 0.4689 - recall: 0.2620 - val_AUC: 0.8610 - val_accuracy: 0.4888 - val_loss: 3.5602 - val_precision: 0.5167 - val_recall: 0.4650
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 417ms/step - AUC: 0.9611 - accuracy: 0.7434 - loss: 1.1468 - precision: 0.7735 - recall: 0.7206 - val_AUC: 0.8572 - val_accuracy: 0.4700 - val_loss: 3.7709 - val_precision: 0.5084 - val_recall: 0.4525
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 423ms/step - AUC: 0.9892 - accuracy: 0.8549 - loss: 0.5224 - precision: 0.8784 - recall: 0.8336 - val_AUC: 0.8586 - val_accuracy: 0.5088 - val_loss: 3.8201 - val_precision: 0.5359 - val_recall: 0.4850
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 403ms/step - AUC: 0.9963 - accuracy: 0.9337 - loss: 0.2438 - precision: 0.9445 - recall: 0.9235 - val_AUC: 0.8555 - val_accuracy: 0.4900 - val_loss: 3.8613 - val_precision: 0.5094 - val_recall: 0.4750
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 429ms/